In [1]:
import d3rlpy
from d3rlpy.algos import COMBO
from sklearn.model_selection import train_test_split
import gymnasium as gym
import numpy as np
import encoders

Compiling /home/arcak-lab/.local/share/virtualenvs/symmetry-cs285-taxvxdSo/lib/python3.8/site-packages/mujoco_py/cymj.pyx because it depends on /home/arcak-lab/.local/share/virtualenvs/symmetry-cs285-taxvxdSo/lib/python3.8/site-packages/Cython/Includes/cpython/type.pxd.
[1/1] Cythonizing /home/arcak-lab/.local/share/virtualenvs/symmetry-cs285-taxvxdSo/lib/python3.8/site-packages/mujoco_py/cymj.pyx


INFO:root:running build_ext
INFO:root:building 'mujoco_py.cymj' extension
INFO:root:creating /home/arcak-lab/.local/share/virtualenvs/symmetry-cs285-taxvxdSo/lib/python3.8/site-packages/mujoco_py/generated/_pyxbld_2.1.2.14_38_linuxgpuextensionbuilder
INFO:root:creating /home/arcak-lab/.local/share/virtualenvs/symmetry-cs285-taxvxdSo/lib/python3.8/site-packages/mujoco_py/generated/_pyxbld_2.1.2.14_38_linuxgpuextensionbuilder/temp.linux-x86_64-cpython-38
INFO:root:creating /home/arcak-lab/.local/share/virtualenvs/symmetry-cs285-taxvxdSo/lib/python3.8/site-packages/mujoco_py/generated/_pyxbld_2.1.2.14_38_linuxgpuextensionbuilder/temp.linux-x86_64-cpython-38/home
INFO:root:creating /home/arcak-lab/.local/share/virtualenvs/symmetry-cs285-taxvxdSo/lib/python3.8/site-packages/mujoco_py/generated/_pyxbld_2.1.2.14_38_linuxgpuextensionbuilder/temp.linux-x86_64-cpython-38/home/arcak-lab
INFO:root:creating /home/arcak-lab/.local/share/virtualenvs/symmetry-cs285-taxvxdSo/lib/python3.8/site-packages

## Params

In [2]:
seed = 1
d3rlpy.seed(seed)
use_gpu = True
# prepare environment
env = gym.make("InvertedPendulum-v4")
eval_env = gym.make("InvertedPendulum-v4")
env.reset(seed=seed)
eval_env.reset(seed=seed)

(array([ 0.00023643,  0.00900927, -0.00711681,  0.00897299]), {})

## Create dataset

In [ ]:
actor_encoder = d3rlpy.models.encoders.DefaultEncoderFactory(dropout_rate=0.2)
# setup algorithm
sac = d3rlpy.algos.SAC(
    batch_size=256,
    actor_encoder_factory=actor_encoder,
    actor_learning_rate=3e-4,
    critic_learning_rate=3e-4,
    temp_learning_rate=3e-4,
    use_gpu=use_gpu
)

# prepare utilities
buffer = d3rlpy.online.buffers.ReplayBuffer(maxlen=1000000, env=env)

# start training
sac.fit_online(
    env,
    buffer,
    eval_env=eval_env,
    n_steps=100000,
    n_steps_per_epoch=1000,
    update_interval=1,
    update_start_step=1000,
    tensorboard_dir='tensorboard_logs'
)

In [ ]:
# export replay buffer as MDPDataset
dataset = buffer.to_mdp_dataset()

# save MDPDataset
dataset.dump('d3rlpy_data/inverted_pendulum2.h5')

## Load the dataset

In [3]:
dataset = d3rlpy.dataset.MDPDataset.load('d3rlpy_data/inverted_pendulum2.h5')

In [4]:
train_episodes, test_episodes = train_test_split(dataset, random_state=seed)

## Dynamics learning

In [6]:
def inverted_pendulum_project(x):
    return x[:, 1:]
projection_size = 3

In [7]:
encoder_factory = encoders.SymmetryEncoderFactory(project=inverted_pendulum_project, projection_size=projection_size)
dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics(learning_rate=1e-4, use_gpu=True, state_encoder_factory=encoder_factory)
#dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics(learning_rate=1e-4, use_gpu=True) # Baseline

Using SymmetryEncoderFactory


In [7]:
# same as algorithms
dynamics.fit(train_episodes,
             eval_episodes=test_episodes,
             n_epochs=100,
             scorers={
                'observation_error': d3rlpy.metrics.scorer.dynamics_observation_prediction_error_scorer,
                'reward_error': d3rlpy.metrics.scorer.dynamics_reward_prediction_error_scorer,
                'variance': d3rlpy.metrics.scorer.dynamics_prediction_variance_scorer,
             },
            tensorboard_dir='tensorboard_logs/dynamics',
            experiment_name='SeparateStateAndRewardEncoders')

2023-10-11 16:01:13 [debug    ] RoundIterator is selected.
2023-10-11 16:01:13 [info     ] Directory is created at d3rlpy_logs/SeparateStateAndRewardEncoders_20231011160113
2023-10-11 16:01:13 [warning  ] Skip building models since they're already built.
2023-10-11 16:01:13 [info     ] Parameters are saved to d3rlpy_logs/SeparateStateAndRewardEncoders_20231011160113/params.json params={'action_scaler': None, 'batch_size': 100, 'discrete_action': False, 'gamma': 1.0, 'generated_maxlen': 100000, 'learning_rate': 0.0001, 'n_ensembles': 5, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0.0001, 'amsgrad': False}, 'real_ratio': 1.0, 'reward_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'reward_scaler': None, 'scaler': None, 'state_encoder_factory': {'type': 'symmetry', 'params': {'hidden_units': [256, 256], 'activation': 'relu', 'use_batch_norm':

Epoch 1/100:   0%|          | 0/714 [00:00<?, ?it/s]

2023-10-11 16:01:42 [info     ] SeparateStateAndRewardEncoders_20231011160113: epoch=1 step=714 epoch=1 metrics={'time_sample_batch': 0.0005488235409520253, 'time_algorithm_update': 0.0357434352238973, 'loss': -32.93809812075617, 'time_step': 0.03645618723220184, 'observation_error': 0.016063637931700202, 'reward_error': 0.0016804140969261861, 'variance': 0.01215117613131028} step=714
2023-10-11 16:01:42 [info     ] Model parameters are saved to d3rlpy_logs/SeparateStateAndRewardEncoders_20231011160113/model_714.pt


Epoch 2/100:   0%|          | 0/714 [00:00<?, ?it/s]

2023-10-11 16:02:12 [info     ] SeparateStateAndRewardEncoders_20231011160113: epoch=2 step=1428 epoch=2 metrics={'time_sample_batch': 0.0005533832127974481, 'time_algorithm_update': 0.0370452494180503, 'loss': -41.99179640804686, 'time_step': 0.03776338420996145, 'observation_error': 0.01028619149651415, 'reward_error': 0.0006694518923929904, 'variance': 0.004676957080329532} step=1428
2023-10-11 16:02:12 [info     ] Model parameters are saved to d3rlpy_logs/SeparateStateAndRewardEncoders_20231011160113/model_1428.pt


...

Epoch 99/100:   0%|          | 0/714 [00:00<?, ?it/s]

2023-10-11 16:49:13 [info     ] SeparateStateAndRewardEncoders_20231011160113: epoch=99 step=70686 epoch=99 metrics={'time_sample_batch': 0.0004933885499542835, 'time_algorithm_update': 0.040260149317295275, 'loss': -86.04313476947175, 'time_step': 0.040914698141295704, 'observation_error': 0.01615742872760815, 'reward_error': 8.8927554587419e-06, 'variance': 0.007027707547621536} step=70686
2023-10-11 16:49:13 [info     ] Model parameters are saved to d3rlpy_logs/SeparateStateAndRewardEncoders_20231011160113/model_70686.pt


Epoch 100/100:   0%|          | 0/714 [00:00<?, ?it/s]

2023-10-11 16:49:47 [info     ] SeparateStateAndRewardEncoders_20231011160113: epoch=100 step=71400 epoch=100 metrics={'time_sample_batch': 0.0005305554686474199, 'time_algorithm_update': 0.04220393391884342, 'loss': -86.33843726203555, 'time_step': 0.04288713838539872, 'observation_error': 0.016058845319678435, 'reward_error': 2.7430494585063786e-06, 'variance': 0.006247049604075671} step=71400
2023-10-11 16:49:47 [info     ] Model parameters are saved to d3rlpy_logs/SeparateStateAndRewardEncoders_20231011160113/model_71400.pt


[(1,
  {'time_sample_batch': 0.0005488235409520253,
   'time_algorithm_update': 0.0357434352238973,
   'loss': -32.93809812075617,
   'time_step': 0.03645618723220184,
   'observation_error': 0.016063637931700202,
   'reward_error': 0.0016804140969261861,
   'variance': 0.01215117613131028}),
 (2,
  {'time_sample_batch': 0.0005533832127974481,
   'time_algorithm_update': 0.0370452494180503,
   'loss': -41.99179640804686,
   'time_step': 0.03776338420996145,
   'observation_error': 0.01028619149651415,
   'reward_error': 0.0006694518923929904,
   'variance': 0.004676957080329532}),
 (3,
  {'time_sample_batch': 0.0005800857597372445,
   'time_algorithm_update': 0.041098010974103996,
   'loss': -49.00794673266531,
   'time_step': 0.0418581545185976,
   'observation_error': 0.012040702134374115,
   'reward_error': 0.00039915916356946386,
   'variance': 0.0093081758324502}),
 (4,
  {'time_sample_batch': 0.0003020255839457365,
   'time_algorithm_update': 0.027733295571570303,
   'loss': -55.

In [8]:
def experiment_dynamics_training(dataset, n_runs, experiment_name, seed=1, use_gpu=True):
    for i in range(n_runs):
        for encoder_factory in ['default', 'symmetry']:
            # use the same seeds for default and symmetric runs
            train_episodes, test_episodes = train_test_split(dataset, random_state=seed+i)
            if encoder_factory == 'symmetry':
                encoder_factory = encoders.SymmetryEncoderFactory(project=inverted_pendulum_project, projection_size=projection_size)
            dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics(learning_rate=1e-4, use_gpu=use_gpu, encoder_factory=encoder_factory)
            dynamics.fit(train_episodes,
                 eval_episodes=test_episodes,
                 n_epochs=100,
                 scorers={
                    'observation_error': d3rlpy.metrics.scorer.dynamics_observation_prediction_error_scorer,
                    'reward_error': d3rlpy.metrics.scorer.dynamics_reward_prediction_error_scorer,
                    'variance': d3rlpy.metrics.scorer.dynamics_prediction_variance_scorer,
                 },
                tensorboard_dir='tensorboard_logs/dynamics',
                experiment_name=experiment_name)

In [9]:
experiment_dynamics_training(dataset=dataset, n_runs=5, experiment_name="exp_2", use_gpu=True)

2023-10-13 17:01:38 [warning  ] Unused arguments are passed.   encoder_factory=default
2023-10-13 17:01:38 [debug    ] RoundIterator is selected.
2023-10-13 17:01:38 [info     ] Directory is created at d3rlpy_logs/exp_2_20231013170138
2023-10-13 17:01:38 [debug    ] Building models...
2023-10-13 17:01:39 [debug    ] Models have been built.
2023-10-13 17:01:39 [info     ] Parameters are saved to d3rlpy_logs/exp_2_20231013170138/params.json params={'action_scaler': None, 'batch_size': 100, 'discrete_action': False, 'gamma': 1.0, 'generated_maxlen': 100000, 'learning_rate': 0.0001, 'n_ensembles': 5, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0.0001, 'amsgrad': False}, 'real_ratio': 1.0, 'reward_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'reward_scaler': None, 'scaler': None, 'state_encoder_factory': {'type': 'default', 'params': {'activ

Epoch 1/100:   0%|          | 0/714 [00:00<?, ?it/s]

2023-10-13 17:01:56 [info     ] exp_2_20231013170138: epoch=1 step=714 epoch=1 metrics={'time_sample_batch': 0.00024600289448970506, 'time_algorithm_update': 0.020046680915255508, 'loss': -21.43590343306076, 'time_step': 0.02036452527139701, 'observation_error': 0.03968955877891613, 'reward_error': 0.001707299197937943, 'variance': 0.03643012516257281} step=714
2023-10-13 17:01:56 [info     ] Model parameters are saved to d3rlpy_logs/exp_2_20231013170138/model_714.pt


Epoch 2/100:   0%|          | 0/714 [00:00<?, ?it/s]

2023-10-13 17:02:13 [info     ] exp_2_20231013170138: epoch=2 step=1428 epoch=2 metrics={'time_sample_batch': 0.00025558838991223933, 'time_algorithm_update': 0.020309349401992242, 'loss': -32.409940701572836, 'time_step': 0.02063505889988747, 'observation_error': 0.0155723516961093, 'reward_error': 0.0015964518172782195, 'variance': 0.01095754946517424} step=1428
2023-10-13 17:02:13 [info     ] Model parameters are saved to d3rlpy_logs/exp_2_20231013170138/model_1428.pt


...

2023-10-13 21:27:01 [info     ] exp_2_20231013210132: epoch=98 step=72814 epoch=98 metrics={'time_sample_batch': 0.0002666331556892652, 'time_algorithm_update': 0.017917011178839416, 'loss': -85.5319157339843, 'time_step': 0.01824887828903917, 'observation_error': 0.01873565433977044, 'reward_error': 5.47965909432192e-06, 'variance': 0.005346745695133894} step=72814
2023-10-13 21:27:01 [info     ] Model parameters are saved to d3rlpy_logs/exp_2_20231013210132/model_72814.pt


Epoch 99/100:   0%|          | 0/743 [00:00<?, ?it/s]

2023-10-13 21:27:16 [info     ] exp_2_20231013210132: epoch=99 step=73557 epoch=99 metrics={'time_sample_batch': 0.0002698834138154021, 'time_algorithm_update': 0.01788323724574709, 'loss': -87.03253068577393, 'time_step': 0.018217657648859242, 'observation_error': 0.01634418893539933, 'reward_error': 2.794779464639418e-06, 'variance': 0.006282450404897598} step=73557
2023-10-13 21:27:16 [info     ] Model parameters are saved to d3rlpy_logs/exp_2_20231013210132/model_73557.pt


Epoch 100/100:   0%|          | 0/743 [00:00<?, ?it/s]

2023-10-13 21:27:32 [info     ] exp_2_20231013210132: epoch=100 step=74300 epoch=100 metrics={'time_sample_batch': 0.00025939909317612166, 'time_algorithm_update': 0.017948818399318937, 'loss': -85.92791411244532, 'time_step': 0.01827312510555878, 'observation_error': 0.017525640980577833, 'reward_error': 2.9611627507947294e-06, 'variance': 0.006151873845335789} step=74300
2023-10-13 21:27:32 [info     ] Model parameters are saved to d3rlpy_logs/exp_2_20231013210132/model_74300.pt


## Load Dynamics

In [ ]:
# load trained dynamics model
dynamics_model_path = "d3rlpy_logs/ProbabilisticEnsembleDynamics_20231002230632"
dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics.from_json(dynamics_model_path + '/params.json')
dynamics.load_model(dynamics_model_path + '/model_31542.pt')

## Train Offline RL Algorithm

In [ ]:
encoders = d3rlpy.models.encoders.DefaultEncoderFactory(dropout_rate=0.2)
# give COMBO as the generator argument.
combo = COMBO(dynamics=dynamics, critic_encoder_factory=encoders, actor_encoder_factory=encoders,
              use_gpu=use_gpu)

In [ ]:
combo.fit(dataset = train_episodes, eval_episodes=test_episodes, n_steps=100000, n_steps_per_epoch=1000, tensorboard_dir="tensorboard_logs",
         scorers={
            'environment': d3rlpy.metrics.scorer.evaluate_on_environment(eval_env)
        })

## Load the Policy

In [ ]:
trained_policy = COMBO()
# initialize with dataset
trained_policy.build_with_dataset(dataset)
# Load entire model parameters.
trained_policy.load_model('d3rlpy_logs/COMBO_20230929153035/model_53000.pt')

## See the policy running

In [ ]:
scorer = d3rlpy.metrics.scorer.evaluate_on_environment(eval_env, render=True)
mean_episode_return = scorer(trained_policy)